In [37]:
import os
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import json

MODELO

In [38]:
BASE_PATH = "../data/processed"
TRAIN_PATHS = [
    "X_train_con_outliers.csv",
    "X_train_sin_outliers.csv",
    "X_train_con_outliers_norm.csv",
    "X_train_sin_outliers_norm.csv",
    "X_train_con_outliers_scal.csv",
    "X_train_sin_outliers_scal.csv"
]
TEST_PATHS = [
    "X_test_con_outliers.csv",
    "X_test_sin_outliers.csv",
    "X_test_con_outliers_norm.csv",
    "X_test_sin_outliers_norm.csv",
    "X_test_con_outliers_scal.csv",
    "X_test_sin_outliers_scal.csv"
]

# Leer los datasets de entrenamiento y prueba
TRAIN_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TRAIN_PATHS]
TEST_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TEST_PATHS]

# Leer las etiquetas (target) de entrenamiento y prueba
y_train = pd.read_csv(os.path.join(BASE_PATH, "y_train.csv")).values.ravel()
y_test = pd.read_csv(os.path.join(BASE_PATH, "y_test.csv")).values.ravel()

# Definir el número de características que deseas seleccionar
k = int(len(pd.read_csv(f"../data/processed/{TRAIN_PATHS[0]}").columns) * 0.35)  # Puedes cambiar este valor según el número de características que desees seleccionar

# Crear una lista para almacenar los resultados
model_results = []

# Crear un diccionario para almacenar todas las características seleccionadas
all_selected_features = {}

for index, dataset in enumerate(TRAIN_DATASETS):
    print(f"Procesando dataset {index + 1}")

    # Selección de características usando SelectKBest con f_regression (para regresión)
    selector = SelectKBest(f_regression, k=k)
    X_train_selected = selector.fit_transform(dataset, y_train)  # Ajuste y transformación para el conjunto de entrenamiento
    X_test_selected = selector.transform(TEST_DATASETS[index])  # Transformación para el conjunto de prueba

    # Almacenar las características seleccionadas en el diccionario
    selected_features = dataset.columns[selector.get_support()].tolist()  # Obtener los nombres de las características seleccionadas
    all_selected_features[TRAIN_PATHS[index]] = selected_features  # Usar el nombre del archivo como clave

    # Crear y entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train_selected, y_train)

    # Predicciones en los datos de entrenamiento y prueba
    y_train_pred = model.predict(X_train_selected)
    y_test_pred = model.predict(X_test_selected)

    # Calcular el MSE y R² para los datos de entrenamiento
    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred)

    # Calcular el MSE y R² para los datos de prueba
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)

    # Almacenar los resultados en la lista, usando el nombre del archivo como "dataset"
    model_results.append(
       {
        "dataset_TRAIN": TRAIN_PATHS[index],  # Usar el nombre del archivo del dataset
        "MSE_train": mse_train,  # Guardar mse_train como número (no como string)
        "r2_train": r2_train,  # Guardar r2_train como número (no como string)
        "dataset_TEST": TEST_PATHS[index],  # Usar el nombre del archivo del dataset
        "MSE_test": mse_test,  # Guardar mse_test como número (no como string)
        "r2_test": r2_test,  # Guardar r2_test como número (no como string)
        "feat_sel": f"k{k}"
    }
    )

# Convertir la lista de resultados en un DataFrame
df_results = pd.DataFrame(model_results)

# Guardar los resultados 
with open(os.path.join(f"../models/model_results_k_{k}.json"), 'w') as json_file:
    json.dump(model_results, json_file, indent=4)

# Guardar las características seleccionadas en un solo archivo JSON
with open(os.path.join(f"../models/selected_features_k_{k}.json"), 'w') as json_file:
    json.dump(all_selected_features, json_file, indent=4)

Procesando dataset 1
Procesando dataset 2
Procesando dataset 3
Procesando dataset 4
Procesando dataset 5
Procesando dataset 6


RESULTADOS

In [39]:
# Crear un DataFrame a partir de los resultados previos
df_results = pd.DataFrame(model_results)

# Calcular la raíz cuadrada del MSE (convertirlo a RMSE)
df_results["RMSE_train"] = np.sqrt(df_results["MSE_train"])
df_results["RMSE_test"] = np.sqrt(df_results["MSE_test"])

# Reordenar las columnas para mostrar solo RMSE y R²
df_results = df_results[["RMSE_train", "RMSE_test", "r2_train", "r2_test"]]

# Imprimir los resultados para revisar
print("Resultados de todos los datasets:")
print(df_results)

# Seleccionar el mejor dataset según el RMSE en el conjunto de prueba (menor RMSE)
best_dataset_index = df_results["RMSE_test"].idxmin()  # Encuentra el índice del menor RMSE en el conjunto de prueba
best_dataset = df_results.iloc[best_dataset_index]  # Obtiene la fila completa con el mejor dataset

# Imprimir el mejor dataset
print("\nEl mejor dataset seleccionado según el menor RMSE en el conjunto de prueba:")
print(best_dataset)

# Obtener el nombre del archivo del mejor dataset
best_train_dataset = TRAIN_PATHS[best_dataset_index]  # El nombre del dataset de entrenamiento correspondiente
best_test_dataset = TEST_PATHS[best_dataset_index]  # El nombre del dataset de prueba correspondiente

print(f"\nEl mejor dataset de entrenamiento es: {best_train_dataset}")
print(f"El mejor dataset de prueba es: {best_test_dataset}")


Resultados de todos los datasets:
     RMSE_train     RMSE_test  r2_train   r2_test
0   2337.000346   2276.731879  0.999516  0.998581
1  89411.741620  43971.568616  0.292242  0.470859
2   2337.000346   2276.731879  0.999516  0.998581
3  89411.741620  43971.568616  0.292242  0.470859
4   2337.000346   2276.731879  0.999516  0.998581
5  89411.741620  43971.568616  0.292242  0.470859

El mejor dataset seleccionado según el menor RMSE en el conjunto de prueba:
RMSE_train    2337.000346
RMSE_test     2276.731879
r2_train         0.999516
r2_test          0.998581
Name: 2, dtype: float64

El mejor dataset de entrenamiento es: X_train_con_outliers_norm.csv
El mejor dataset de prueba es: X_test_con_outliers_norm.csv


In [40]:
# Definir el indice del mejor dataset
best_dataset_idx = 2

LASSO

In [41]:
from sklearn.linear_model import Lasso

alpha = 0.1
lasso_model = Lasso(alpha = alpha, max_iter = 5000, random_state = 42)

lasso_model.fit(TRAIN_DATASETS[best_dataset_idx], y_train)

y_pred_train = lasso_model.predict(TRAIN_DATASETS[best_dataset_idx])
y_pred_test = lasso_model.predict(TEST_DATASETS[best_dataset_idx])

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print(mse_train)
print(r2_train)
print(mse_test)
print(r2_test)


# df_results = pd.DataFrame(results)
# # Calcular RMSE
# df_results["train_rmse"] = np.sqrt(df_results["train_mse"])
# df_results["test_rmse"] = np.sqrt(df_results["test_mse"])
# # Seleccionar columnas relevantes para mostrar en tabla
# df_results = df_results[["train_rmse", "test_rmse", "train_r2", "test_r2"]]
# # Mostrar tabla
# print(df_results)

# # Imprimir los resultados
# print(f"MSE: {mse}")
# print(f"R2 Score: {r2}")
# print("")
# print("Coefficients:", lasso_model.coef_)

# # Crear un Diccionario con los resultados
# lasso_results = {
#     "Model": ["Lasso"],
#     "Alpha": [alpha],
#     "MSE": [mse],
#     "R2 Score": [r2],
# }

# # Almacenar el resultado de lasso en un archivo
# with open(os.path.join(f"../data/results/lasso_results.json"), 'w') as json_file:
#     json.dump(lasso_results, json_file, indent=4)

3761950.193899409
0.9996669503513365
4606285.509463127
0.9987393962995226


c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.730e+09, tolerance: 2.837e+09
  model = cd_fast.enet_coordinate_descent(


RIDGE

In [42]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge_model = Ridge(alpha = 0.5, random_state= 42)

param_grid = {
    "alpha": [0.01, 0.1, 1, 10, 100],
    "max_iter": [100, 300, 1000, 2000, 4000, 8000, 12000],
    "tol": np.linspace(0.0001, 0.01, 100),
}

grid_ridge = GridSearchCV(estimator = ridge_model, param_grid = param_grid, scoring = 'r2', cv = 5, n_jobs = -1)

ridge_model.fit(TRAIN_DATASETS[best_dataset_idx], y_train)

y_pred_train = ridge_model.predict(TRAIN_DATASETS[best_dataset_idx])
y_pred_test = ridge_model.predict(TEST_DATASETS[best_dataset_idx])


results_ridge = []

results_ridge.append({
        "train_r2": r2_score(y_train, y_pred_train),
        "train_mse": mean_squared_error(y_train, y_pred_train),
        "test_r2": r2_score(y_test, y_pred_test),
        "test_mse": mean_squared_error(y_test, y_pred_test),
    })

df_ridge = pd.DataFrame(results_ridge)

print(df_ridge)

   train_r2     train_mse   test_r2      test_mse
0  0.999675  3.671906e+06  0.998846  4.216755e+06


ELASTICNET

In [43]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

elastic_model = ElasticNet(alpha = 0.5, random_state= 42)

param_grid_elastic = {
    "alpha": [0.01, 0.1, 1, 10],
    "l1_ratio": [0.1, 0.5],
    "max_iter": [100, 300, 1000, 2000],
    "tol": np.linspace(0.0001, 0.01, 10),
}

grid_elastic = GridSearchCV(estimator = elastic_model, param_grid = param_grid_elastic, scoring = 'r2', cv = 5, n_jobs = -1)

grid_elastic.fit(TRAIN_DATASETS[best_dataset_idx], y_train)

y_pred_train = grid_elastic.predict(TRAIN_DATASETS[best_dataset_idx])
y_pred_test = grid_elastic.predict(TEST_DATASETS[best_dataset_idx])


results_elastic = []

results_elastic.append({
        "train_r2": r2_score(y_train, y_pred_train),
        "train_mse": mean_squared_error(y_train, y_pred_train),
        "test_r2": r2_score(y_test, y_pred_test),
        "test_mse": mean_squared_error(y_test, y_pred_test),
    })

df_elastic = pd.DataFrame(results_elastic)

print(df_elastic)

   train_r2     train_mse   test_r2      test_mse
0  0.999263  8.329956e+06  0.997394  9.522615e+06
